In [1]:
import pandas as pd
import numpy as np
from itertools import repeat
from scipy.sparse import coo_matrix
import scipy
import scipy.signal

tl=pd.read_csv('../../Rscripts/data/90360AAAE_PR35_M65705_11_005.03.csv',skiprows=[1,2,3], low_memory=False, error_bad_lines=False)

In [2]:
def countnnfails(df):
    row = np.array(df['x coord'])
    col = np.array(df['y coord'])
    data = np.array(np.logical_and(df['BIN']>349,df['BIN']<500))
    data = data.astype(int)
    spm = coo_matrix((data, (row, col))).toarray()
    
    return(spm)

In [3]:
tlu=tl[['x coord','y coord',"BIN"]].groupby(['x coord','y coord']).head(1)

In [4]:
sp = countnnfails(tlu)

In [5]:
np.bincount(np.ravel(sp))

array([11217,    93])

In [6]:
import scipy.signal

In [45]:
kernel = np.array([[1,1,1],[1,0,1],[1,1,1]])
nncount = scipy.signal.convolve2d(sp, kernel, 'same')

In [50]:
np.bincount(np.ravel(nncount))

array([10720,   472,    90,    21,     6,     1])

In [105]:
nndfx = np.transpose(np.nonzero(nncount))[:,0]
nndfx[np.isnan(nndfx)] = 0
nndfy = np.transpose(np.nonzero(nncount))[:,1]
nndfy[np.isnan(nndfy)] = 0
nndfnncount = np.ravel(nncount)
nndfnncount = nndfnncount[np.nonzero(nndfnncount)]

In [106]:
nndfnncount[np.nonzero(nndfnncount)].size

590

In [107]:
nndfx.size

590

In [108]:
nndf = pd.DataFrame({'x' : pd.Series(nndfx),
                     'y' : pd.Series(nndfy),
                     'nncount' : pd.Series(nndfnncount)})

In [109]:
nndf = nndf.fillna(0)
nndf['x'] = nndf['x'].astype('int32')
nndf['y'] = nndf['y'].astype('int32')

In [110]:
nndf['nncount'].value_counts()

1    472
2     90
3     21
4      6
5      1
Name: nncount, dtype: int64

In [111]:
nndf.dtypes

nncount    int64
x          int32
y          int32
dtype: object

In [117]:
nndf.loc[np.logical_and(nndf['x']==79,nndf['y']==67),:]

,nncount,x,y
282,2,79,67
